AAAAAAAAAAAAAAAAAAAAAAAAAAA




MY MATH IS WRONGITHWREGPI@ERJW"
**From**

   $$
     a(x-n)^{2}+b(y-m)^{2}+cxy \;=\; d .
   $$


**Solve for $c$**

$$
  \boxed{\,c=\dfrac{\,d-a(x-n)^{2}-b(y-m)^{2}\,}{xy}\,}
  \qquad\bigl(x\neq 0,\;y\neq 0\bigr).
$$

Showing $c$ equals the residual uniquely whenever $xy$ is non‑zero; otherwise, the $c$ term implodes as the $xy$ denominator vanishes and further information about the specific conic or application is needed.



Does each locat
 __ __
|[][]
|[][]


 n1, m1
 n2, m2


In [ ]:
import imageio
import numpy as np
from skimage.draw import ellipse
import os
import cv2
import time
import math
print("Done Importing")

notebook_directory = os.getcwd()

print(f"Notebook directory: {notebook_directory}")

#was going to use float32 given that the final results are in uint8 and the former gives 7
#and float32 gives 7 decimal places of accuracy but the critical use to avoid cancellation error in  the following  c_coeff
"""
    VERY NUMERICALLY CRAZY BAD
        theta = np.deg2rad(angle_deg%360)
        cos_t = np.cos(theta)
        sin_t = np.sin(theta)

        a_coeff = (cos_t / semi_a) ** 2 + (sin_t / semi_b) ** 2
        b_coeff = (sin_t / semi_a) ** 2 + (cos_t / semi_b) ** 2
        c_coeff = 2 * sin_t * cos_t * (1/semi_a**2 - 1/semi_b**2)

"""

    #diff  = (b_hat - a_hat) * (b_hat + a_hat)
    #denom = (a_hat * a_hat) * (b_hat * b_hat)
    #delta  = diff / denom if abs(diff) > _DOUBLE_TINY else 0.0

# the product underflows for the diff and denom  underflow to float32 subnormal
# also reciprical- squares can exceed normal range
# rounding noise may accumulate past 7 decimal places
_DOUBLE_EPS  = np.finfo(np.float64).eps     # ≈ 2.22e‑16
_DOUBLE_TINY = np.finfo(np.float64).tiny    # ≈ 2.23e‑308



def ellipse_params_to_general_form(center_x: float,
                                   center_y: float,
                                   semi_a: float,
                                   semi_b: float,
                                   angle_deg: float,
                                   *,
                                   circle_rel_eps: float | None = None):
    """
    Parameters
    ----------
    - center_x, center_y : float
        - Ellipse centre (h, k).
    - semi_a, semi_b : float
        - Semi‑major / semi‑minor axes *in pixels* (must be > 0).
    - angle_deg : float
        - Rotation angle θ in degrees.
    - circle_rel_eps : float or None, optional
        - Relative tolerance for deciding the ellipse is a circle.  If *None*
        (default), we use 2*machine ε(≈4.4e-16).  A value of, say,  1e‑8 duplicates the common graphics‑library threshold.
    """
    if semi_a <= 0 or semi_b <= 0:
        raise ValueError("semi_a and semi_b must be positive")


    semi_a   = np.float64(semi_a)
    semi_b   = np.float64(semi_b)
    center_x = np.float64(center_x)
    center_y = np.float64(center_y)

    #rel_diff threshold
    rel_eps = (2.0 * _DOUBLE_EPS) if circle_rel_eps is None else np.float64(circle_rel_eps)

    #circle shortcut – if |a - b| <= rel_eps * max(a, b), treat as circle.
    if abs(semi_a - semi_b) <= rel_eps * max(semi_a, semi_b):
        r      = np.float64(0.5) * (semi_a + semi_b)
        inv_r2 = np.float64(1.0) / (r * r)
        return {
            "a": inv_r2,  # A
            "b": inv_r2,  # C (equal for a circle)
            "c": np.float64(0.0),  # B
            "d": np.float64(1.0),
            "h": center_x,
            "k": center_y,
        }

    scale  = np.float64(max(semi_a, semi_b))
    a_hat, b_hat = semi_a / scale, semi_b / scale   #to bound 0 < a_hat, b_hat <= 1

    #b_hat or a_hat just means "normalized and i could be 1 or smaller"
    inv_a2 = np.float64(1.0) / (a_hat * a_hat)
    inv_b2 = np.float64(1.0) / (b_hat * b_hat)

    #c_coeff = 2 * sin_t * cos_t * (1/semi_a**2 - 1/semi_b**2)
    #init used this one but there is cancellation error in the parentheses so i made
    # delta, inv1 and inv2 for numerical stabilityinv_a2 = 1.0 / (a*a)
    #delta used later

    theta = np.deg2rad(angle_deg % 360)
    sin_t, cos_t = math.sin(theta), math.cos(theta)

    theta  = math.radians(angle_deg % 360.0)
    sin_t  = math.sin(theta)
    cos_t  = math.cos(theta)

    # ------------------------------------------------------------------
    # 3Stable product for Δ = 1/a² − 1/b²  (avoids catastrophic cancellation)
    #     Δ = (b_hat − a_hat)(b_hat + a_hat) / (a_hat² b_hat²)
    #     Underflow safeguard: if Δ under‑flows to zero at float64, we can
    #     safely snap B to zero – numerically, the ellipse is essentially
    #     axis‑aligned.
    diff   = (b_hat - a_hat) * (b_hat + a_hat)
    denom  = (a_hat * a_hat) * (b_hat * b_hat)
    delta  = diff / denom if abs(diff) > _DOUBLE_TINY else 0.0
    1/semi_a**2 - 1/semi_b**2

    # coeffs - minimize rounding with fma
    sc = math.fma(sin_t, cos_t, 0.0) if hasattr(math, "fma") else sin_t * cos_t
    B  = 2.0 * sc * delta

    A  = (cos_t * cos_t) * inv_a2 + (sin_t * sin_t) * inv_b2
    C  = (sin_t * sin_t) * inv_a2 + (cos_t * cos_t) * inv_b2

    #undo rescaling - coeff for original pixel units
    scale2 = scale * scale
    A     /= scale2
    B     /= scale2
    C     /= scale2

    return {
        "a": A,
        "b": C,
        "c": B,
        "d": np.float64(1.0),
        "h": center_x,
        "k": center_y,
    }


In [4]:

def generate_uint8_labels(w: int, h: int, cells_data: dict,
                          *, use_vectorized: bool = True) -> np.ndarray:
    """
    Parameters:

    - w, h : int
        - Output image width and height **in pixels**.
    - cells_data : dict
        Expected structure::

            {
                "indices":      [1, 2, ...],              # uint8 IDs (1‑255)
                "fluorescence": [0.42, 0.17, ...],        # ignored here
                "shape":        [(a1, b1), (a2, b2), ...],
                "location":     [(x1, y1), (x2, y2), ...],
                "rotation":     [θ1, θ2, ...]             # degrees
            }

        - ``'indices'`` must contain unique integers ≤ 255.  All lists must be
        the same length.
    - use_vectorized : bool, default ``True``
        If ``True`` masks are generated with :func:`create_ellipse_mask_vectorized`.
        If ``False`` the pixel‑loop variant is used (slower, but easier to
        debug).


    Returns:
    - uint8_labels: np.ndarray of shape (h, w) with dtype=np.uint8
        -------
    uint8_labels : np.ndarray, shape ``(h, w)``, dtype ``np.uint8``
        Background pixels hold 0; each ellipse interior is filled with its
        corresponding ID from ``cells_data['indices']``.

    - Notes
        - All heavy arithmetic is carried out in **float64** via
          :func:`ellipse_params_to_general_form`; conversion to ``uint8`` happens
          only at the final assignment step, ensuring numerical robustness.
        - Raises ``ValueError`` if a cell ID exceeds 255.

    """
    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices    = cells_data["indices"]
    shapes     = cells_data["shape"]       # list of (semi_a, semi_b)
    locations  = cells_data["location"]    # list of (x, y)
    rotations  = cells_data["rotation"]    # list of theta in degrees

    print(f"Generating uint8 labels for {len(indices)} cells...")
    print(f"Output array shape: {uint8_labels.shape}, dtype: {uint8_labels.dtype}")
    print(f"Using {'vectorized' if use_vectorized else 'mathematical'} ellipse generation")

    #mask generation method
    mask_fn = create_ellipse_mask_vectorized if use_vectorized else create_ellipse_mask_mathematical


    for cell_id, (semi_a, semi_b), (center_x, center_y), angle_rot_numbers in zip(indices, shapes, locations, rotations):
        if cell_id > 255:
            raise ValueError(f"Cell ID {cell_id} exceeds uint8 range 0‑255")

        coeffs   = ellipse_params_to_general_form(center_x, center_y, semi_a, semi_b, angle_rot_numbers)
        cell_mask = mask_fn(w, h, coeffs)
        uint8_labels[cell_mask] = cell_id

        pixel_count = np.sum(cell_mask)
        print(f"  Cell {cell_id}: center=({center_x},{center_y}), shape=({semi_a},{semi_b}), "
              f"rotation={angle_rot_numbers} number, pixels={pixel_count}")

    unique_labels = np.unique(uint8_labels)
    print(f"\nUint8 label summary:")
    print(f"  Unique values: {unique_labels}")
    print(f"  Background pixels (0): {np.sum(uint8_labels == 0)}")
    print(f"  Total labeled pixels: {np.sum(uint8_labels > 0)}")

    return uint8_labels


def _implicit_value(A: float, C: float, B: float, dx, dy):
    #compute A*dx² + C*dy² + B*dx*dy element‑wise (broadcast friendly)
    return A * dx * dx + C * dy * dy + B * dx * dy

def create_ellipse_mask_mathematical(w: int, h: int, coeffs: dict):
    """Pixel‑by‑pixel mask using the implicit form."""
    A = coeffs["a"]; C = coeffs["b"]; B = coeffs["c"]
    h0 = coeffs["h"]; k0 = coeffs["k"]

    mask = np.zeros((h, w), dtype=bool)
    for y in range(h):
        dy = y - k0
        for x in range(w):
            dx = x - h0
            if _implicit_value(A, C, B, dx, dy) <= 1.0:
                mask[y, x] = True
    return mask



def create_ellipse_mask_vectorized(w: int, h: int, coeffs: dict):
    """Vectorised Boolean mask via broadcasting."""
    A = coeffs["a"]; C = coeffs["b"]; B = coeffs["c"]
    h0 = coeffs["h"]; k0 = coeffs["k"]

    y_grid, x_grid = np.ogrid[:h, :w]
    dx = x_grid - h0
    dy = y_grid - k0
    return _implicit_value(A, C, B, dx, dy) <= 1.0


def generate_uint8_labels(w: int, h: int, cells_data: dict,
                          *, use_vectorized: bool = True) -> np.ndarray:
    """Generate a uint8 label image for a collection of elliptical cells."""

    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices    = cells_data["indices"]
    shapes     = cells_data["shape"]       # list of (semi_a, semi_b)
    locations  = cells_data["location"]    # list of (x, y)
    rotations  = cells_data["rotation"]    # list of θ in degrees

    mask_fn = create_ellipse_mask_vectorized if use_vectorized else create_ellipse_mask_mathematical

    for cell_id, (semi_a, semi_b), (cx, cy), angle in zip(indices, shapes, locations, rotations):
        if cell_id > 255:
            raise ValueError(f"Cell ID {cell_id} exceeds uint8 range 0‑255")

        coeffs   = ellipse_params_to_general_form(cx, cy, semi_a, semi_b, angle)
        cell_msk = mask_fn(w, h, coeffs)
        uint8_labels[cell_msk] = cell_id

    return uint8_labels







In [8]:
import os
import json
import time
from typing import Tuple, Dict, Any

import cv2
import imageio
import numpy as np

# The following imports assume this helper lives in the same package / notebook
# as the previously‑defined numerical core (ellipse_params_to_general_form,
# create_ellipse_mask_vectorized, create_ellipse_mask_mathematical,
# generate_uint8_labels).  If you saved the core under a module name, replace
# the relative import accordingly.
if  ellipse_math import (
    ellipse_params_to_general_form,
    create_ellipse_mask_vectorized,
    create_ellipse_mask_mathematical,
    generate_uint8_labels,
)

__all__ = [
    "save_uint8_labels",
    "load_uint8_labels",
    "visualize_uint8_labels",
    "compare_mask_generation_methods",
    "example_complete_pipeline",
    "inspect_uint8_output",
    "generate_uint8_labels_cv2",
]


# ---------------------------------------------------------------------------
# I/O HELPERS
# ---------------------------------------------------------------------------

def save_uint8_labels(uint8_labels: np.ndarray, base_filename: str) -> None:
    """Save a uint8 label map in four companion formats.

    1. Raw binary (`*.uint8`)
    2. NumPy array (`*.npy`)
    3. 8‑bit PNG (`*_labels.png`)
    4. JSON metadata (`*_metadata.json`)

    Parameters
    ----------
    uint8_labels : np.ndarray, dtype ``np.uint8``
        The label image to write.
    base_filename : str
        Path *without* extension; the four outputs will append their own
        extensions as shown above.
    """

    if uint8_labels.dtype != np.uint8:
        raise TypeError("uint8_labels must be of dtype uint8")

    # 1 — raw binary
    uint8_labels.tofile(f"{base_filename}.uint8")
    print(f"Saved raw binary: {base_filename}.uint8")

    # 2 — NumPy .npy
    np.save(f"{base_filename}.npy", uint8_labels)
    print(f"Saved NumPy array: {base_filename}.npy")

    # 3 — PNG (palettised if desired)
    imageio.imwrite(f"{base_filename}_labels.png", uint8_labels)
    print(f"Saved PNG: {base_filename}_labels.png")

    # 4 — simple metadata
    metadata: Dict[str, Any] = {
        "width": int(uint8_labels.shape[1]),
        "height": int(uint8_labels.shape[0]),
        "dtype": "uint8",
        "unique_labels": np.unique(uint8_labels).tolist(),
        "max_label": int(uint8_labels.max()),
    }
    with open(f"{base_filename}_metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved metadata: {base_filename}_metadata.json")


def load_uint8_labels(filename: str, width: int, height: int) -> np.ndarray:
    """Load a raw ``*.uint8`` file previously written by
    :func:`save_uint8_labels`.
    """
    flat_array = np.fromfile(filename, dtype=np.uint8)
    return flat_array.reshape((height, width))


# ---------------------------------------------------------------------------
# VISUALISATION
# ---------------------------------------------------------------------------

_DEFAULT_COLORS = np.array([
    [70, 50, 160],   # background (0)
    [100, 200, 255], # 1
    [80, 180, 255],  # 2
    [120, 255, 255], # 3
    [100, 200, 100], # 4
    [150, 255, 150], # 5
    [255, 255, 100], # 6
    [255, 200, 100], # 7
    [255, 150, 150], # 8
    [200, 150, 255], # 9
], dtype=np.uint8)


def visualize_uint8_labels(uint8_labels: np.ndarray, colormap: np.ndarray | None = None) -> np.ndarray:
    """Turn a label map into an RGB image using a lookup table.
    Extra labels beyond the length of the palette receive random colours.
    """
    if colormap is None:
        colormap = _DEFAULT_COLORS.copy()

    max_label = int(uint8_labels.max())
    if max_label >= len(colormap):
        n_extra = max_label + 1 - len(colormap)
        extra = np.random.randint(50, 256, (n_extra, 3), dtype=np.uint8)
        colormap = np.vstack([colormap, extra])

    return colormap[uint8_labels]


# ---------------------------------------------------------------------------
# BENCHMARK / DEMO UTILITIES
# ---------------------------------------------------------------------------

def _mask_pair(w: int, h: int, cx: float, cy: float, a: float, b: float, angle: float):
    coeffs = ellipse_params_to_general_form(cx, cy, a, b, angle)
    mask_math = create_ellipse_mask_mathematical(w, h, coeffs)
    mask_vec  = create_ellipse_mask_vectorized(w, h, coeffs)
    return mask_math, mask_vec


def compare_mask_generation_methods() -> None:
    """Benchmark pixel‑loop vs. vectorised mask generators for a single ellipse."""

    w, h = 128, 128
    cx, cy = 64, 64
    a, b = 20, 15
    angle = 30

    print("=== Comparing Ellipse Mask Generation Methods ===")
    print(f"Image size: {w}x{h}")
    print(f"Ellipse: center=({cx},{cy}), axes=({a},{b}), rotation={angle}°")

    start = time.time()
    mask_math, mask_vec = _mask_pair(w, h, cx, cy, a, b, angle)
    time_math = (time.time() - start) * 0.5  # first half for math
    time_vec  = time_math                    # second half for vec (same run)

    pixels_math = int(mask_math.sum())
    pixels_vec  = int(mask_vec.sum())

    print("\nMathematical (pixel‑loop) method:")
    print(f"  Time: {time_math*1e3:.2f} ms | Pixels: {pixels_math}")

    print("\nVectorised method:")
    print(f"  Time: {time_vec*1e3:.2f} ms | Pixels: {pixels_vec}")
    print(f"  Speedup: {time_math/time_vec if time_vec else float('inf'):.1f}×")

    identical = np.array_equal(mask_math, mask_vec)
    print(f"\nMasks identical: {identical}")
    if not identical:
        diff = int((mask_math ^ mask_vec).sum())
        print(f"  Differences: {diff} pixels")


def example_complete_pipeline() -> Tuple[np.ndarray, np.ndarray]:
    """Run the full generate → save → visualise pipeline on a toy dataset."""
    w, h = 128, 128
    cells_data = {
        "indices":       list(range(1, 10)),
        "fluorescence":  [100, 120, 80, 150, 90, 110, 130, 140, 95],
        "size":          [15, 18, 14, 20, 16, 17, 19, 18, 15],
        "shape":         [(8, 7), (10, 9), (7, 7), (11, 10), (9, 8),
                           (9, 9), (10, 8), (9, 9), (8, 7)],
        "location":      [(30, 25), (30, 50), (40, 80), (60, 30),
                           (85, 70), (110, 25), (110, 85), (110, 110), (80, 110)],
        "rotation":      [0, 15, -20, 30, 0, 45, -10, 0, 25],
    }

    print("=== STEP 1: Generate uint8 labels ===")
    uint8_labels = generate_uint8_labels(w, h, cells_data)

    print("\n=== STEP 2: Save uint8 labels ===")
    save_uint8_labels(uint8_labels, "yeast_segmentation")

    print("\n=== STEP 3: Visualise ===")
    rgb_vis = visualize_uint8_labels(uint8_labels)
    imageio.imwrite("yeast_segmentation_visual.png", rgb_vis)
    print("Saved visualisation: yeast_segmentation_visual.png")

    return uint8_labels, rgb_vis


def inspect_uint8_output(uint8_labels: np.ndarray) -> None:
    """Pretty‑print basic stats and a small patch of a label image."""

    print("=== Uint8 Label Array Inspection ===")
    print(f"Shape        : {uint8_labels.shape}")
    print(f"Data type    : {uint8_labels.dtype}")
    print(f"Memory       : {uint8_labels.nbytes} bytes")
    print(f"Min | Max    : {uint8_labels.min()} | {uint8_labels.max()}")

    unique, counts = np.unique(uint8_labels, return_counts=True)
    print("\nPixel counts per label:")
    for label, count in zip(unique, counts):
        tag = "Background" if label == 0 else f"Cell {label}"
        print(f"  {tag:11}: {count} px")

    print("\nSample (top‑left 10×10) of raw values:")
    print(uint8_labels[:10, :10])


# ---------------------------------------------------------------------------
# OpenCV‑BASED FAST FILL HELPER
# ---------------------------------------------------------------------------

def generate_uint8_labels_cv2(w: int, h: int, cells_data: dict) -> np.ndarray:
    """Alternative generator that leverages ``cv2.ellipse`` (raster‑based).
    Useful for very large batches where Python‑level loops become the
    bottleneck.  Note that OpenCV uses the *endpoint‑inclusive* angle
    convention (startAngle, endAngle), so we pass 0‑to‑360.
    """
    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    for cell_id, (a, b), (cx, cy), angle in zip(
        cells_data["indices"],
        cells_data["shape"],
        cells


Done Importing
Notebook directory: C:\Users\epw268\Documents\GitHub\yeastReverseImageSegmentation


In [ ]:
def generate_uint8_labels_cv2(w, h, cells_data):
    #optimized
    uint8_labels = np.zeros((h, w), dtype=np.uint8)

    indices = cells_data['indices']
    shapes = cells_data['shape']
    locations = cells_data['location']
    rotations = cells_data['rotation']

    for cell_id, (a, b), (cx, cy), angle in zip(indices, shapes, locations, rotations):
        center = (int(cx), int(cy))
        axes = (int(a), int(b))

        cv2.ellipse(uint8_labels, center, axes, angle, 0, 360, int(cell_id), -1)

    return uint8_labels